In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import sys
import os.path
import matplotlib.pyplot as plt
import matplotlib as mpl
#mpl.style.use('v2.0')
plt.style.use('ggplot')

import numpy as np
import pandas as pd
from tqdm import tqdm

#import os, sys
sys.path.append(os.path.dirname("../"))
sys.path.append(os.path.dirname("../../"))

from plugins.FaxDistiller import Distill

In [2]:
from mltools.tfrio import TFRecordsIO

/cfs/klemming/nobackup/b/bobau/ToolBox/TestEnv/Anaconda3/envs/ml_pax/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
raw_file = "../data/FakeWaveform_raw/XENON1T-0-000000000-000000999-000001000.zip"
truth_file = "../data/FakeWaveform_truth.csv"
instructions_file = "../data/FakeWaveform.csv"

data_path = "/cfs/klemming/nobackup/j/jcalven/wavenet_data/"

### Load FAX raw data and distill down into useful parts

In [4]:
distiller = Distill(raw_file, truth_file, instructions_file)
distiller.load()

100%|██████████| 1000/1000 [00:01<00:00, 737.81it/s]


### The way to retrieve distilled data

In [ ]:
### Distill.get() returns iterator of distilled data for the event (objects of the Condensate() class)
event = distiller.get(1).__next__()
event.get_event().head()

In [ ]:
print(event.__doc__)

In [ ]:
def format_event_data(waveforms, instructions):
    
    return dict(data = waveforms.values.astype(int),
                x = instructions["x"].values[0],
                y = instructions["y"].values[0],
                depth = instructions["depth"].values[0],
                s1_photons = instructions["s1_photons"].values[0],
                s2_electrons = instructions["s2_electrons"].values[0],
                event_number = instructions["instruction"].values[0])

### Loop over events and store distilled data for each event

In [ ]:
#events = []
#waveforms_in_channels = []
#event_truths = []
#event_instructions = []

save_to_dir = "/cfs/klemming/nobackup/j/jcalven/wavenet_data/"
fax_file = "XENON1T-0-000000000-000000999-000001000"

n_events = 1000
counter = 1

tfr_io = TFRecordsIO()
tfr_io.open_tfwriter_stream(filename=data_path + fax_file + "_{}_events.tfr".format(n_events))

for i, distilled_events in tqdm(enumerate(distiller.get(n_events))):
    
    #events.append(distilled_events.get_event())  # Retrieve pandas.DataFrame of event information
    #waveforms_in_channels.append(distilled_events.get_waveforms_in_channels())  # Retrieve pandas.DataFrame 
                                                                                # of per channel reconstructed 
                                                                                # waveforms in the event
    #event_truths.append(distilled_events.get_event_truth())  # Retrieve pandas.DataFrame of PAX truth information for event
    #event_instructions.append(distilled_events.get_event_instructions())
    
    waveforms_in_channels = distilled_events.get_waveforms_in_channels()
    event_instructions = distilled_events.get_event_instructions()
    
    # Preprocess the data for feeding to TFRecords
    event_data = format_event_data(waveforms_in_channels, event_instructions)
    
    # Write data to TFRecords files
    #tfr_io.write(filename=data_path + "test_10_events.tfr", **event_data)
    tfr_io.write(**event_data)
    
    #df = pd.read_pickle(filename)
    #key = os.path.basename(filename) # or choose another name
    #if i+1%100 == 0 or i == 0:
    #    print("Writing to hdf file {}...".format(counter))
    #    file_it = 100 * counter
    #    output_filename = save_to_dir + fax_file + "_" + str(file_it) + ".h5"
    #    counter += 1
    #waveforms_in_channels.to_hdf(output_filename, "waveforms_{}".format(i))
    #event_instructions.to_hdf(output_filename, "instructions_{}".format(i))
    
tfr_io.close_tfwriter_stream()

In [ ]:
event_truths[0]

In [ ]:
event_instructions[0]

In [ ]:
for i in range(5):
    waveforms_in_channels[i].to_pickle(
        "/cfs/klemming/nobackup/j/jcalven/wavenet_data/test_{}.pkl".format(i))

In [ ]:
wf_array = waveforms_in_channels[0].values
#wf_array = wf_array[15600:15700,:]# 120:130]
wf_array_flat = wf_array.flatten()

In [ ]:
wf_array.flatten().shape

In [ ]:
np.allclose(wf_array, wf_array.reshape(wf_array.shape))

In [ ]:
wf_array

In [ ]:
wf_array.shape

In [ ]:
wf_array.max()

In [ ]:
z_min, z_max = wf_array.min(), np.abs(wf_array).max()
print(z_min, z_max)

fig = plt.figure()#figsize=(20,5))
plt.pcolormesh(wf_array, vmin=z_min, vmax=z_max)

In [ ]:
# Concat all dataframes
#events_concat = pd.concat(events)
waveforms_in_channels_concat = pd.concat(waveforms_in_channels)
#event_truths_concat = pd.concat(event_truths)

#waveforms_in_channels_concat.head()

In [ ]:
### Plot of waveforms for each pmt summed up

sum_waveforms = waveforms_in_channels[0].drop("event_number", axis=1).sum(axis=1)

fig = plt.figure(figsize=(14,6))
ax = plt.subplot(111)
p1 = ax.plot(sum_waveforms.index, sum_waveforms)
ax.set_xlabel("Time [ns]", size=14)
ax.set_ylabel("ADC counts", size=14)
plt.tight_layout()
# fig.savefig("recon_waveform_example.png", dpi=240)

In [ ]:
len(waveforms_in_channels[0].columns)

In [ ]:
dfc.empty()

In [ ]:
maxlist = []
for wf in waveforms_in_channels:
    df = wf[wf > 0.] #waveforms_in_channels[0]
    #df = df[df > 0.]
    for i, channel in enumerate(df):
        dfc = df[channel].dropna()
        if not dfc.empty:
            maxlist.append(dfc.max())

max(maxlist)


    #df = waveforms_in_channels[0][channel]#["channel_5"]
    #df.dropna(inplace=True)
    #df = 
    #print(df.loc[df > 0.].max())
    #max_counts[i] = df.loc[df > 0.].max()
    #print(df.loc[df > 0.].max())

#pd.Series(max_counts).dropna().max()
#df[df > 0.]["channel_0"].dropna()
#df.loc[df > 0.]

In [ ]:
len(maxlist)

In [ ]:
max_counts = np.zeros(len(waveforms_in_channels[0].columns))
for i, channel in enumerate(waveforms_in_channels[0]):
    df = waveforms_in_channels[0][channel]#["channel_5"]
    df.dropna(inplace=True)
    #df = 
    #print(df.loc[df > 0.].max())
    max_counts[i] = df.loc[df > 0.].max()
    #print(df.loc[df > 0.].max())

pd.Series(max_counts).dropna().max()
#df#.hist()
#df.max()
#df.describe()
#df.hist()

In [ ]:
df.loc[df > 0.].min()

In [ ]:
df.plot()